In [1]:
import qsharp

Preparing Q# environment...
.

In [2]:
qsharp.packages.add("Microsoft.Quantum.Chemistry")

Adding package Microsoft.Quantum.Chemistry.

In [3]:
qsharp.reload()

Reloading workspace.

In [4]:
from Microsoft.Quantum.Samples.Chemistry.SimpleVQE import Test

In [6]:
Test.simulate(theta1=0.001, theta2=-0.001, theta3=0.001, termType=0, nHam=0, nOp=0)

Total ham terms: 4
Total measurement operations: 1


1

In [7]:
Test.simulate(theta1=0.001, theta2=-0.001, theta3=0.001, termType=1, nHam=0, nOp=0)

Total ham terms: 6
Total measurement operations: 1


0

In [8]:
Test.simulate(theta1=0.001, theta2=-0.001, theta3=0.001, termType=3, nHam=0, nOp=0)

Total ham terms: 1
Total measurement operations: 8


0

In [21]:
!az quantum execute --target-id ionq.simulator -- --theta1 0.001 --theta2 -0.001 --theta3 0.001 --term-type 0 --n-ham 0 --n-op 0

Command group 'quantum' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
....
{
  "Histogram": [
    "0",
    2.4999997919999996e-07,
    "1",
    0.9999997500000226
  ]
}


In [105]:
import subprocess
import json

def run_vqe(term_type, n_ham, n_op, shots, target="ionq.simulator"):
    cmd = f"az quantum execute --target-id {target} -- --theta1 0.001 --theta2 -0.001 --theta3 0.001 --term-type {term_type} --n-ham {n_ham} --n-op {n_op} --shots {shots}"
    run_cmd = subprocess.run([cmd], check=True, shell=True, capture_output=True)
    result = run_cmd.stdout
    result = json.loads(run_cmd.stdout.lstrip(b"."))
    return result, run_cmd

In [106]:
hamiltonian = (
    [
        ([0], [0.17120128499999998]),
        ([1], [0.17120128499999998]),
        ([2], [-0.222796536]),
        ([3], [-0.222796536])
    ],
    [
        ([0, 1], [0.1686232915]),
        ([0, 2], [0.12054614575]),
        ([0, 3], [0.16586802525]),
        ([1, 2], [0.16586802525]),
        ([1, 3], [0.12054614575]),
        ([2, 3], [0.1743495025])
    ],
    [],
    [
        ([0, 1, 2, 3], [0.0, -0.0453218795, 0.0, 0.0453218795])
    ]
)

In [129]:
target = "ionq.simulator"
results = []
shots = 100
for term_type in [0,1,3]:
    if term_type == 0:
        n_hams = range(4)
    elif term_type == 1:
        n_hams = range(6)
    elif term_type == 3:
        n_hams = [0]
    for n_ham in n_hams:
        n_ops = [0] if term_type != 3 else range(4)
        for n_op in n_ops:
            results.append(run_vqe(term_type, n_ham, 0, shots, target))

In [130]:
estimate_energy(hamiltonian, results)

-0.5356664127610464

In [131]:
target = "ionq.simulator"
results = []
shots = 100
for term_type in [0,1,3]:
    if term_type == 0:
        n_hams = range(4)
    elif term_type == 1:
        n_hams = range(6)
    elif term_type == 3:
        n_hams = [0]
    for n_ham in n_hams:
        n_ops = [0] if term_type != 3 else range(4)
        for n_op in n_ops:
            results.append(run_vqe(term_type, n_ham, 0, shots, target))

In [134]:
estimate_energy(hamiltonian, results)

-0.5356664127610464

In [95]:
def estimate_energy(hamiltonian, results):
    termExpectation = [a["Histogram"][1] for a, _ in results]
    coeff = [__c for t in hamiltonian for _t, _c in t for __c in _c]
    return sum([(2. * _t - 1.) * _c for _t, _c in zip(termExpectation, coeff)])

In [96]:
estimate_energy(hamiltonian, results)

-1.017848070739604

In [60]:
[a for a, _ in results]

[{'Histogram': ['0', 2.4999997919999996e-07, '1', 0.9999997500000226]},
 {'Histogram': ['0', 2.2499968124999997e-06, '1', 0.9999977500031892]},
 {'Histogram': ['0', 0.9999997500000226, '1', 2.4999997919999996e-07]},
 {'Histogram': ['0', 0.9999977500031892, '1', 2.2499968124999997e-06]},
 {'Histogram': ['0', 0.9999975000043351, '1', 2.4999956667e-06]},
 {'Histogram': ['0', 4.999998333999999e-07, '1', 0.9999995000001684]},
 {'Histogram': ['0', 2.4999956667e-06, '1', 0.9999975000043351]},
 {'Histogram': ['0', 2.4999956667e-06, '1', 0.9999975000043351]},
 {'Histogram': ['0', 4.4999835e-06, '1', 0.9999955000165016]},
 {'Histogram': ['0', 0.9999975000043351, '1', 2.4999956667e-06]},
 {'Histogram': ['0', 1.0000000000000018]},
 {'Histogram': ['0', 1.0000000000000018]},
 {'Histogram': ['0', 1.0000000000000018]},
 {'Histogram': ['0', 1.0000000000000018]}]

In [ ]:
(2. * termExpectation - 1.) * coeff

In [27]:
import qsharp.azure

In [28]:
resource_id = "/subscriptions/9006b025-1a81-4f7c-9111-582e121509ac/resourcegroups/azure-quantum-guenp/providers/Microsoft.Quantum/Workspaces/azure-quantum-01"
qsharp.azure.connect(resourceId=resource_id)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RLMVK4XTS to authenticate.

Connected to Azure Quantum workspace azure-quantum-01 in location westus.


[{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 102},
 {'id': 'ionq.simulator', 'current_availability': 'Available', 'average_queue_time': 0},
 {'id': 'honeywell.hqs-lt-1.0', 'current_availability': 'Degraded', 'average_queue_time': 0},
 {'id': 'honeywell.hqs-lt-1.0-apival', 'current_availability': 'Available', 'average_queue_time': 0}]

In [29]:
qsharp.azure.target("ionq.simulator")

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RXNAYXYWA to authenticate.

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


{'id': 'ionq.simulator', 'current_availability': 'Available', 'average_queue_time': 0}

In [12]:
result1 = qsharp.azure.execute(
    Test,
    theta1=0.001,
    theta2=-0.001,
    theta3=0.001,
    termType=0,
    shots=100,
    nHam=0,
    nOp=0,
    jobName="Simulate Hydrogen using VQE"
)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RR6EFCWTY to authenticate.

Submitting Microsoft.Quantum.Samples.Chemistry.SimpleVQE.Test to target ionq.simulator...
Job successfully submitted for 100 shots.
   Job name: Simulate Hydrogen using VQE
   Job ID: 50f3389d-38fa-4303-aa3b-0784ed5ff619
Waiting up to 30 seconds for Azure Quantum job to complete...
[5:51:08 PM] Current job status: Succeeded


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RCXY9GVFK to authenticate.

In [19]:
Test.simulate(theta1=0.001, theta2=-0.001, theta3=0.001, termType=0, nHam=0, nOp=0)

Total ham terms: 4
Total measurement operations: 1
0


1

In [30]:
Test.simulate(theta1=0.001, theta2=-0.001, theta3=0.001, termType=1, nHam=0, nOp=0)

0

In [32]:
Test.simulate(theta1=0.001, theta2=-0.001, theta3=0.001, termType=3, nHam=0, nOp=0)

1

In [22]:
qsharp.azure.target("ionq.qpu")

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R667Y5V5A to authenticate.

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 33}

In [23]:
result1 = qsharp.azure.execute(
    Test,
    theta1=0.001,
    theta2=-0.001,
    theta3=0.001,
    termType=0,
    shots=100,
    nHam=0,
    nOp=0,
    jobName="Simulate Hydrogen using VQE"
)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R7RSJMRRA to authenticate.

Submitting Test to target ionq.qpu...
Job successfully submitted for 100 shots.
   Job name: Simulate Hydrogen using VQE
   Job ID: f4464075-3854-4b47-adaa-e95f06a8a474
Waiting up to 30 seconds for Azure Quantum job to complete...
[5:15:31 PM] Current job status: Waiting
[5:15:37 PM] Current job status: Waiting
[5:15:42 PM] Current job status: Waiting
[5:15:47 PM] Current job status: Waiting
[5:15:52 PM] Current job status: Waiting


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FG55TVW47 to authenticate.

In [11]:
print(f'Azure Quantum service execution result: {result1}')

Azure Quantum service execution result: {'0': 2.4999997919999996e-07, '1': 0.9999997500000226}


In [12]:
qsharp.azure.target("ionq.qpu")

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RTQH6AYM9 to authenticate.

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 50}

In [13]:
result2 = qsharp.azure.execute(
    Test,
    theta1=0.001,
    theta2=-0.001,
    theta3=0.001,
    termType=0,
    shots=100,
    nHam=0,
    nOp=0,
    jobName="Simulate Hydrogen using VQE"
)
print(f'Azure Quantum service execution result: {result2}')

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RGBHQD3KZ to authenticate.

Submitting Test to target ionq.qpu...
Job successfully submitted for 100 shots.
   Job name: Simulate Hydrogen using VQE
   Job ID: cb123e27-7467-4844-a5bb-3fbbbe73a4e7
Waiting up to 30 seconds for Azure Quantum job to complete...
[4:58:14 PM] Current job status: Waiting
[4:58:19 PM] Current job status: Waiting
[4:58:24 PM] Current job status: Succeeded


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RR5PHG6QF to authenticate.

Azure Quantum service execution result: {'0': 0.01, '1': 0.99}


In [13]:
import subprocess

In [ ]:
subprocess.run